In [24]:
import tkinter as tk
from PIL import Image, ImageTk

window = tk.Tk()
window.title("Sudoku")
icon = ImageTk.PhotoImage(file= 'icon.png')
window.iconphoto(True, icon)

canvas = tk.Canvas(window, bg="white", height=550, width=550)
canvas.grid(row= 1, column= 1)

case_taille = 50

for ligne in range(9):
    for colonne in range(9):
        for case in range(9):
            x1 = 50 + colonne * case_taille
            y1 = 50 + ligne * case_taille
            x2 = x1 + case_taille
            y2 = y1 + case_taille
            rect = canvas.create_rectangle(x1, y1, x2, y2, fill="cornflower blue", outline= "black", width= 1)

window.mainloop()